In [5]:
import pandas as pd

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}
df = pd.read_csv(
    "legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)
df.tail()

,last_name,first_name,birthday,gender,type,state,party
11970,Garrett,Thomas,1972-03-27,M,rep,VA,Republican
11971,Handel,Karen,1962-04-18,F,rep,GA,Republican
11972,Jones,Brenda,1959-10-24,F,rep,MI,Democrat
11973,Marino,Tom,1952-08-15,M,rep,PA,Republican
11974,Jones,Walter,1943-02-10,M,rep,NC,Republican


In [6]:
n_by_state = df.groupby("state")["last_name"].count()
n_by_state.head(10)

state
AK     16
AL    206
AR    117
AS      2
AZ     48
CA    361
CO     90
CT    240
DC      2
DE     97
Name: last_name, dtype: int64

In [8]:
df.groupby(["state", "gender"])["last_name"].count()

state  gender
AK     F           0
       M          16
AL     F           3
       M         203
AR     F           5
                ... 
WI     M         196
WV     F           1
       M         119
WY     F           2
       M          38
Name: last_name, Length: 116, dtype: int64

In [9]:
df.groupby(["gender", "state"])["last_name"].count()

gender  state
F       AK         0
        AL         3
        AR         5
        AS         0
        AZ         3
                ... 
M       VT       115
        WA        90
        WI       196
        WV       119
        WY        38
Name: last_name, Length: 116, dtype: int64

In [10]:
df.groupby(["state", "gender"], as_index=False)["last_name"].count()

,state,gender,last_name
0,AK,F,NaN
1,AK,M,16.0
2,AL,F,3.0
3,AL,M,203.0
4,AR,F,5.0
...,...,...,...
111,WI,M,196.0
112,WV,F,1.0
113,WV,M,119.0
114,WY,F,2.0


In [11]:
df = pd.read_csv(
    "airqual.csv",
    parse_dates=[["Date", "Time"]],
    na_values=[-200],
    usecols=["Date", "Time", "CO(GT)", "T", "RH", "AH"]
).rename(
    columns={
        "CO(GT)": "co",
        "Date_Time": "tstamp",
        "T": "temp_c",
        "RH": "rel_hum",
        "AH": "abs_hum",
    }
).set_index("tstamp")

df.head()
    

,co,temp_c,rel_hum,abs_hum
tstamp,,,,
2004-03-10 18:00:00,2.6,13.6,48.9,0.758
2004-03-10 19:00:00,2.0,13.3,47.7,0.726
2004-03-10 20:00:00,2.2,11.9,54.0,0.750
2004-03-10 21:00:00,2.2,11.0,60.0,0.787
2004-03-10 22:00:00,1.6,11.2,59.6,0.789


In [12]:
day_names = df.index.day_name()
type(day_names)
day_names[:10]

Index(['Wednesday', 'Wednesday', 'Wednesday', 'Wednesday', 'Wednesday',
       'Wednesday', 'Thursday', 'Thursday', 'Thursday', 'Thursday'],
      dtype='object', name='tstamp')

In [13]:
df.groupby(day_names)["co"].mean()

tstamp
Friday       2.543
Monday       2.017
Saturday     1.861
Sunday       1.438
Thursday     2.456
Tuesday      2.382
Wednesday    2.401
Name: co, dtype: float64

In [14]:
hr = df.index.hour
df.groupby([day_names, hr])["co"].mean().rename_axis(["dow", "hr"])

dow        hr
Friday     0     1.936
           1     1.609
           2     1.172
           3     0.887
           4     0.823
                 ...  
Wednesday  19    4.147
           20    3.845
           21    2.898
           22    2.102
           23    1.938
Name: co, Length: 168, dtype: float64

In [15]:
df.groupby([df.index.year, df.index.quarter])["co"].agg(
...     ["max", "min"]
... ).rename_axis(["year", "quarter"])

max  min
year quarter           
2004 1         8.1  0.3
     2         7.3  0.1
     3         7.5  0.1
     4        11.9  0.1
2005 1         8.7  0.1
     2         5.0  0.3